In [140]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import lxml.html as lh
import regex as re

In [141]:
r = requests.get('https://pokemondb.net/pokedex/all')

In [142]:
print(r.url)
print(r.status_code)
# print(r.content)

https://pokemondb.net/pokedex/all
200


In [143]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pokémon Pokédex: list of Pokémon with stats | Pokémon Database
  </title>
  <link href="https://img.pokemondb.net" rel="preconnect"/>
  <style>
   @font-face{font-family:"Fira Sans";font-style:normal;font-weight:400;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-400.woff2") format("woff2");unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+2000-206F,U+2074,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD}@font-face{font-family:"Fira Sans";font-style:italic;font-weight:400;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-400i.woff2") format("woff2");unicode-range:U+0000-00FF,U+0131,U+0152-0153,U+02BB-02BC,U+02C6,U+02DA,U+02DC,U+2000-206F,U+2074,U+20AC,U+2122,U+2191,U+2193,U+2212,U+2215,U+FEFF,U+FFFD}@font-face{font-family:"Fira Sans";font-style:normal;font-weight:700;font-display:swap;src:url("/static/fonts/fira-sans-v10-latin-600.woff2") format("

In [144]:
print(soup.title)
print(soup.title.name)
print(soup.title.parent.name)

<title>Pokémon Pokédex: list of Pokémon with stats | Pokémon Database</title>
title
head


In [145]:
soup = BeautifulSoup(r.content, 'html.parser')
 
s = soup.find('div', class_='resp-scroll')
content = s.find_all('table')

print(content)

[<table class="data-table sticky-header block-wide" id="pokedex">
<thead>
<tr>
<th class="sorting" data-sort="int"><div class="sortwrap">#</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Name</div></th> <th><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Total</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">HP</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Attack</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Defense</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Atk</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Def</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Speed</div></th> </tr>
</thead>
<tbody>
<tr>
<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><img alt="Bulbasaur" class="img-fixed icon-pkmn" height="42" loading="lazy" src="https://i

In [146]:
soup = BeautifulSoup(r.content, 'html.parser')
 
s = soup.find('div', class_='resp-scroll')
content = s.find('table')
individual = s.find_all(class_="cell-num cell-fixed")

# print(individual)
print(individual[0])

<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><img alt="Bulbasaur" class="img-fixed icon-pkmn" height="42" loading="lazy" src="https://img.pokemondb.net/sprites/sword-shield/icon/bulbasaur.png" width="56"/></span><span class="infocard-cell-data">001</span></td>


In [147]:
doc = lh.fromstring(r.content)
tr_elements = doc.xpath('//tr')
td_elements = doc.xpath('//td')

In [148]:
soup = BeautifulSoup(r.content, "html.parser")
table = soup.find_all(class_ = "data-table sticky-header block-wide")

In [149]:
table1 = table[0]

body = table1.find_all("tr")

table_header = body[0]
table_rows = body[1:]

In [150]:
print("Table header: \n", table_header)
print("\n\n rows: \n", table_rows[0])

Table header: 
 <tr>
<th class="sorting" data-sort="int"><div class="sortwrap">#</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Name</div></th> <th><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Total</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">HP</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Attack</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Defense</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Atk</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Def</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Speed</div></th> </tr>


 rows: 
 <tr>
<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><img alt="Bulbasaur" class="img-fixed icon-pkmn" height="42" loading="lazy" src="https://img.pokemondb.net/sprites/sword-shield/icon/bulbasaur.png" width=

In [151]:
column_names = []
for item in table_header.find_all("th"):
    item = (item.text).rstrip("\n")
    column_names.append(item)
print(column_names)

['#', 'Name', 'Type', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']


In [152]:
all_rows = []
for row_num in range(len(table_rows)):
    individual_row = []
    for row_item in table_rows[row_num].find_all("td"):
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        individual_row.append(aa)
    # append one row to all_rows
    all_rows.append(individual_row)
print(all_rows)

[['001', 'Bulbasaur', 'Grass Poison', '318', '45', '49', '49', '65', '65', '45'], ['002', 'Ivysaur', 'Grass Poison', '405', '60', '62', '63', '80', '80', '60'], ['003', 'Venusaur', 'Grass Poison', '525', '80', '82', '83', '100', '100', '80'], ['003', 'Venusaur Mega Venusaur', 'Grass Poison', '625', '80', '100', '123', '122', '120', '80'], ['004', 'Charmander', 'Fire ', '309', '39', '52', '43', '60', '50', '65'], ['005', 'Charmeleon', 'Fire ', '405', '58', '64', '58', '80', '65', '80'], ['006', 'Charizard', 'Fire Flying', '534', '78', '84', '78', '109', '85', '100'], ['006', 'Charizard Mega Charizard X', 'Fire Dragon', '634', '78', '130', '111', '130', '85', '100'], ['006', 'Charizard Mega Charizard Y', 'Fire Flying', '634', '78', '104', '78', '159', '115', '100'], ['007', 'Squirtle', 'Water ', '314', '44', '48', '65', '50', '64', '43'], ['008', 'Wartortle', 'Water ', '405', '59', '63', '80', '65', '80', '58'], ['009', 'Blastoise', 'Water ', '530', '79', '83', '100', '85', '105', '78'],

In [153]:
#### NEED TO DO SOMETHING ABOUT MEGA EVOLUTION AND ALOLAN AND ALL THE DIFFERENT FORMS

In [154]:
##### DATAFRAME GENERATION

In [155]:
df = pd.DataFrame(data=all_rows,columns=column_names)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


In [156]:
df.shape

(1190, 10)

In [162]:
df.iloc[-1,:]

#                     1008
Name              Miraidon
Type       Electric Dragon
Total                  670
HP                     100
Attack                  85
Defense                100
Sp. Atk                135
Sp. Def                115
Speed                  135
Name: 1189, dtype: object

In [168]:
df.to_csv('pokedex_dataset.csv',sep=',', mode = 'w', index=False)